<h2>개인 구글 드라이브와 colab 연동</h2>

In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-5m9osowk/kobert-tokenizer_808613449240496ea39525a78c9d3375
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-5m9osowk/kobert-tokenizer_808613449240496ea39525a78c9d3375
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=f4fb5f894cc4557a0a835c7f4b5a4be6f16ecbe0af7274328534c7bfc48ca8c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-qw7z7iiy/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
!pip install transformers
!pip install sentencepiece
root_dir = "/gdrive/MyDrive/NLP/week10/"

import sys
sys.path.append(root_dir)

<h2>WSD 모델</h2>

In [ ]:
from transformers import BertPreTrainedModel, BertModel
#BERT 사전학습모델을 사용

class WSD(BertPreTrainedModel):
#BERT 사전학습모델을 상속
    def __init__(self, config):
      #class의 attribute를 선언 및 초기화하는 작업
        super(WSD, self).__init__(config)

        # BERT 모델
        self.bert = BertModel(config)

    def forward(self, input_ids, attention_mask):
      #attention_mask는 Self Attention 범위 지정을 위한 마스크(padding 부분이 아닌 곳에만 attention을 걸기)
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        #기존에 정의한 self.bert 모델에 파라미터를(입력의 token과 attention_mask) 대입해 outputs을 받기

        # BERT 출력 (batch_size, max_length, hidden_size)
        bert_output = outputs[0]
        #우리는 bert 모델의 batch 중에서 맨 마지막 층을 사용할 것
        #즉, output들 중에서 내가 궁금한 '배'라는 word를 원하면 배의 embedding vector를 뽑아서 사용한다는 의
        return bert_output

<h2>데이터 읽고 전처리 하기</h2>

<pre>
<b>1. read_data(file_path)</b>
  "datas.txt" 파일을 읽기 위한 함수
  
  데이터 예시)
    토큰들로 구성된 토큰 시퀀스 \t "배"에 대응하는 토큰 인덱스

    ▁보기 만 ▁해도 ▁배 가 ▁부르 다 .	3
    ▁점 심을 ▁먹 지 ▁못해 ▁배 가 ▁많이 ▁고 팠 다 .	5
    ▁배 ▁한 ▁ 척 이 ▁바다 ▁한 가 운 데 ▁떠 ▁있다 .	0
    ▁그 ▁섬 에는 ▁하루 에 ▁두 ▁번 씩 ▁배 가 ▁들어 온 다 .	8
    ▁나는 ▁ 과 일 ▁중 에서 ▁배 를 ▁가장 ▁좋아 한다 .	6
    ▁사 각 사 각 ▁ 씹 히 는 ▁배 의 ▁맛 이 ▁달 고 ▁시 원 하다 .	8
  
  read_data(file_path)
  args
    file_path : 읽고자 하는 데이터의 경로
  return
    토큰 sequence, "배"에 대응하는 토큰 인덱스를 담고 있는 리스트
    
    출력 예시)
      datas = [ (['▁보기', '만', '▁해도', '▁배', '가', '▁부르', '다', '.'], 3)

                (...),
        
              ]
      
<b>2. convert_data2feature(datas, max_length, tokenizer)</b>
  입력 데이터를 indexing 한 후, padding 추가
  Tensor로 변환
   
  convert_data2feature(datas, max_length, tokenizer)
  args
    datas : 토큰 sequence, "배"에 대응하는 토큰 인덱스를 담고 있는 리스트
    max_length : 입력의 최대 길이
    tokenizer : BERT 토크나이저
  return
    input_ids_features : 입력 문장에 대한 index sequence
    attention_mask_features : padding을 제외한 나머지 실제 토큰 정보를 갖고 있는 sequence
    
  전처리 예시)
    tokens : ['[CLS]', '▁보기', '만', '▁해도', '▁배', '가', '▁부르', '다', '.', '[SEP]']
    input_ids : [2, 2362, 6150, 5002, 2287, 5330, 2432, 5782, 54, 3, ...]
    attention_mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...]
 </pre>


In [ ]:
import torch


def read_data(file_path):
  #입력 문장 데이터가 Bert의 토크나이저에 의해 '_보기 만 _해도'같이 토크나이징 되어 있는데
  #이 토큰 형태를 sentencepiece라고 하고 이것이 bert가 사용하는 토크나이저가 잘라놓은 형태
  #sentencepiece는 BytePairEncoding이라는 방법으로 잘라놓은 것들을 wordPiece라고 하는데 이와 매우 유사한 방식
  #이런 식으로 자른 것을 입력으로 받을 것
  #물론 '_점 심을 +먹 지'처럼 형태소 분석을 하지 않고 통계적으로 wordPiece방식으로 잘라서 이상하게 잘렸는데
  #장점은 세상의 어떤 언어든지 다 이런 방식으로 자를 수 있음.
  #즉, 여러 나라의 언어들을 쉽게 의미 단위로 비슷하게 자를 수 있다는 장점이 있기에 구글이 사용함

  #'▁점 심을 ▁먹 지 ▁못해 ▁배 가 ▁많이 ▁고 팠 다 .    5'을 보면 \t으로 분리되어 뒤에 숫자가 있는데
  #이는 타겟으로 하는 그 단어 즉, embedding을 뽑아보고 싶은 그 단어의 위치를 의미
  #즉, 0부터 시작해 5번째인 '_배'를 명시적으로 의미
    with open(file_path, "r", encoding="utf8") as inFile:
        lines = inFile.readlines()

    datas = []
    for line in lines:
        # 입력 데이터를 \t을 기준으로 분리
        pieces = line.strip().split("\t")

        # 입력 토큰 시퀀스(자른 word의 토큰), 목표 토큰 인덱스(내가 원하는 word 토큰의 위치)
        token_sequence, target_token_index = pieces[0].split(" "), int(pieces[1])

        datas.append((token_sequence, target_token_index))
        #튜플 단위로 묶어서 리스트에 append
    return datas


def convert_data2feature(datas, max_length, tokenizer):
  #datas : 위에서 만든 튜플 단위 리스트
  #max_length : bert가 입력받을 수 있는 최대 길이 (보통 512)
  #tokenizer : senetencePiece 단위로 쪼개주는 토크나이저

    input_ids_features, attention_mask_features = [], []
    #input_ids_features는 input_ids인 embedding된 token을 담기
    #attention_mask_features는 input_ids_features가 유효한 것들에만 1로 마스킹해서 담기

    #(입력 토큰 시퀀스(자른 word의 토큰), 목표 토큰 인덱스(내가 원하는 word 토큰의 위치))
    for token_sequence, target_token_index in datas:

        # CLS, SEP 토큰 추가
        tokens = [tokenizer.cls_token]  #Bert의 입력 처음에는 [CLS] 토큰을 넣어야 함
        tokens += token_sequence        #자른 word의 토큰을 차례대로 대입
        tokens = tokens[:max_length - 1]#너무 입력이 많으면 잘라야 하고 [SEP] 토큰 자리도 준비해야함
        tokens += [tokenizer.sep_token] #마지막은 [SEP] 토큰을 대입

        # word piece들을 대응하는 index로 치환
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        #bert 내의 tokenizer를 그대로 사용해서 토큰을 넣으면 id들을 그 결과로 반환

        # padding을 제외한 실제 데이터 정보를 반영해주기 위한 attention mask
        attention_mask = [1] * len(input_ids)
        #내가 받은 id의 길이만큼 1을 넣고 나머지는 padding으로 0으로 attention_mask를 처리
        #이 방식은 attention_mask에도 동

        # padding 생성
        padding = [tokenizer._convert_token_to_id(tokenizer.pad_token)] * (max_length - len(input_ids))
        padding_for_mask = [0] * (max_length - len(input_ids))

        # padding 추가
        input_ids += padding
        attention_mask += padding_for_mask

        # 변환한 데이터를 각 리스트에 저장
        input_ids_features.append(input_ids)
        attention_mask_features.append(attention_mask)

    # 변환한 데이터를 Tensor 객체에 담아 반환
    input_ids_features = torch.tensor(input_ids_features, dtype=torch.long)
    attention_mask_features = torch.tensor(attention_mask_features, dtype=torch.long)

    return input_ids_features, attention_mask_features

<h2>WSD 모델을 사용하여 문맥에 따라 변하는 단어 벡터 확인</h2>

<pre>
<b>1. read_data(file_path) 함수를 사용하여 입력 데이터 읽기</b>

<b>2. convert_data2feature(datas, max_length, tokenizer) 함수를 사용하여 데이터 전처리</b>

<b>3. WSD 모델을 활용하여 "배"에 대응하는 벡터 표현 추출</b>

<b>4. 서로 다른 문장에서 사용된 "배"에 대응하는 벡터 표현 사이의 유사도 비교</b>
</pre>

In [ ]:
import torch
import operator
import numpy as np

from transformers import BertConfig
#from tokenization_kobert import KoBertTokenizer
from kobert_tokenizer import KoBERTTokenizer

def get_cos_sim(vector_1, vector_2):
  #input_id를 vector한 뒤에 그것으로 코사인 유사도를 내적으로 계산해 반환
  return np.dot(vector_1, vector_2)/(np.linalg.norm(vector_1)*np.linalg.norm(vector_2))

#우리는 training할 것이 아니라 사전학습모델에 넣어 결과만 뽑고 싶기에 test만 정
def test(config):
    # BERT config 객체 생성 : 사전학습모델의 것을 그대로 사용
    bert_config = BertConfig.from_pretrained(pretrained_model_name_or_path=config["pretrained_model_name_or_path"],
                                             cache_dir=config["cache_dir_path"])

    # BERT tokenizer 객체 생성 : tokenizer 객체도 사전학습모델의 것을 그대로 이용
    bert_tokenizer = KoBERTTokenizer.from_pretrained(pretrained_model_name_or_path=config["pretrained_model_name_or_path"],
                                                     cache_dir=config["cache_dir_path"])

    # 데이터 읽기 : sentencePiece로 잘려진 data 그대로 return
    datas = read_data(file_path=config["data_path"])

    # 데이터 전처리 : 받은 data를 넣어서 input의 id와 attention_mask를 받
    input_ids_features, attention_mask_features = convert_data2feature(datas=datas,
                                                                       max_length=config["max_length"],
                                                                       tokenizer=bert_tokenizer)

    # 사전 학습된 BERT 모델 파일로부터 가중치 불러옴
    # 우리가 만든 WSD는 사전학습모델인 BERT를 상속받음
    model = WSD.from_pretrained(pretrained_model_name_or_path=config["pretrained_model_name_or_path"],
                                cache_dir=config["cache_dir_path"], config=bert_config).cuda()

    input_ids_features = input_ids_features.cuda()
    attention_mask_features = attention_mask_features.cuda()

    # 모델 예측 결과
    bert_outputs = model(input_ids_features, attention_mask_features)
    #결과로서 output[0]인 마지막 층의 임베딩 값들이 나옴 : [CLS] ~ [PAD]

    #눈으로 보기 편하게 하기 위해 tensor형태에서 list 형태로 변
    input_ids_features = input_ids_features.cpu().detach().numpy().tolist()
    bert_outputs = bert_outputs.cpu().detach().numpy().tolist()

    # batch 단위로 구성되어 있어 반복문을 통해 하나씩 확인
    word2vec = {}
    # batch 안에 사람의 배, 타는 배, 먹는 배가 각각 2개씩 있는데 거기서 for문으로 하나씩 뽑기
    for batch_index in range(len(bert_outputs)):#0~5번 인덱스까지 6회 반복
        input_tokens = bert_tokenizer.convert_ids_to_tokens(input_ids_features[batch_index])
        bert_output = bert_outputs[batch_index]

        #batch 내에서 인덱스로 입력 토큰 시퀀스 문장인 token_sequence를 받고 내가 원하는 '배'의 token_index를 반환 받기
        token_sequence, target_token_index = datas[batch_index]

        # token_sequence : 입력 토큰 시퀀스 문장으로 변환
        # ['▁보기', '만', '▁해도', '▁배', '가', '▁부르', '다', '.'] -> 보기만 해도 배가 부르다.
        # 이는 그저 보기 좋게 하기 위해서 구성한 구문 & 메서드
        sentence = bert_tokenizer.convert_tokens_to_string(token_sequence)

        # token_sequence 앞에 [CLS] 토큰이 추가되었기 때문에 1 추가
        target_token_index += 1

        target_token = input_tokens[target_token_index]
        # 토큰을 단어로 변경 (_배 -> 배)
        # 이는 그저 보기 좋게 하기 위해서 구성한 구문 & 메서드
        target_word = bert_tokenizer.convert_tokens_to_string([target_token])

        #목표하는 토큰 인덱스를 output 리스트에 대입해 '배'에 해당하는 embedding vector를 반환받기
        target_vector = bert_output[target_token_index]
        # 단어, 단어가 사용된 batch_index, 단어가 사용된 문장
        # 배_1 (보기만 해도 배가 부르다.)
        word2vec["{}_{} ({})".format(target_word, batch_index+1, sentence)] = target_vector
        #word2vec이라는 딕셔너리의 key에는 '배_i'와 그것에 해당하는 '문장'을 넣고
        #그 딕셔너리에 key의 value로 그 target_word의 embedding 값인 target_vector 넣기

    # "배_i"에 대응하는 나머지 모든 벡터 표현들 사이의 유사도 계산
    word_similarity = {}
    for word_1, vector_1 in word2vec.items():
      #딕셔너리의 key와 value를 하나씩 가져와서

        # word_1과 나머지 단어들 사이의 유사도를 담을 리스트 생성
        #즉, 해당 key를 기준으로 다른 key들 간의 유사도를 담을 리스트
        word_similarity[word_1] = []
        for word_2, vector_2 in word2vec.items():
            #같은 토큰이면 건너뛰기
            if word_1 == word_2:
              continue

            # word1과 word2 사이의 코사인 유사도를 계산한 후, 그 결과를 word_similarity 딕셔너리에 저장
            # word_similarity의 key : word1, value : (word2, 유사도)
            #           코드 작성 부분              #
            cos_sim = get_cos_sim(vector_1 = vector_1, vector_2 = vector_2)
            word_similarity[word_1].append((word_2, cos_sim))
            #word_1 기준 word_2~word_6까지 각각의 유사도를 튜플로 엮어 value에 append

    print("\n단어1 (단어1이 사용된 문장) vs 단어2 (단어2가 사용된 문장) -> 유사도\n")
    for word in word_similarity.keys():

        #각 key에 대해 value인 유사도를 기준으로 정렬, reverse=True를 통해 내림차순으로 정렬
        word_similarity[word] = sorted(word_similarity[word], key=operator.itemgetter(1), reverse=True)

        for index in range(len(word_similarity[word])):
            print("{} vs {} -> {}".format(word, word_similarity[word][index][0], round(word_similarity[word][index][1], 4)))
        print()

In [ ]:
import os


if(__name__=="__main__"):
    cache_dir = os.path.join(root_dir, "cache")
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)

    config = {
        "data_path": os.path.join(root_dir, "datas.txt"),
        "cache_dir_path": cache_dir,
        "pretrained_model_name_or_path": "skt/kobert-base-v1",
        "max_length": 30
    }

    test(config=config)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.



단어1 (단어1이 사용된 문장) vs 단어2 (단어2가 사용된 문장) -> 유사도

배_1 (보기만 해도 배가 부르다.) vs 배_2 (점심을 먹지 못해 배가 많이 고팠다.) -> 0.7238
배_1 (보기만 해도 배가 부르다.) vs 배_4 (그 섬에는 하루에 두 번씩 배가 들어온다.) -> 0.6015
배_1 (보기만 해도 배가 부르다.) vs 배_6 (사각사각 씹히는 배의 맛이 달고 시원하다.) -> 0.5815
배_1 (보기만 해도 배가 부르다.) vs 배_5 (나는 과일 중에서 배를 가장 좋아한다.) -> 0.5376
배_1 (보기만 해도 배가 부르다.) vs 배_3 (배 한 척이 바다 한가운데 떠 있다.) -> 0.4839

배_2 (점심을 먹지 못해 배가 많이 고팠다.) vs 배_1 (보기만 해도 배가 부르다.) -> 0.7238
배_2 (점심을 먹지 못해 배가 많이 고팠다.) vs 배_5 (나는 과일 중에서 배를 가장 좋아한다.) -> 0.5074
배_2 (점심을 먹지 못해 배가 많이 고팠다.) vs 배_4 (그 섬에는 하루에 두 번씩 배가 들어온다.) -> 0.5071
배_2 (점심을 먹지 못해 배가 많이 고팠다.) vs 배_6 (사각사각 씹히는 배의 맛이 달고 시원하다.) -> 0.4337
배_2 (점심을 먹지 못해 배가 많이 고팠다.) vs 배_3 (배 한 척이 바다 한가운데 떠 있다.) -> 0.4213

배_3 (배 한 척이 바다 한가운데 떠 있다.) vs 배_4 (그 섬에는 하루에 두 번씩 배가 들어온다.) -> 0.7059
배_3 (배 한 척이 바다 한가운데 떠 있다.) vs 배_6 (사각사각 씹히는 배의 맛이 달고 시원하다.) -> 0.6008
배_3 (배 한 척이 바다 한가운데 떠 있다.) vs 배_5 (나는 과일 중에서 배를 가장 좋아한다.) -> 0.5345
배_3 (배 한 척이 바다 한가운데 떠 있다.) vs 배_1 (보기만 해도 배가 부르다.) -> 0.4839
배_3 (배 한 척이 바다 한가운데 떠 있다.) vs 배_2